In [1]:
import requests
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import ast
pd.options.display.max_columns = 999

In [2]:
def scrap (id):
    url = 'https://brainout.org/user/' + str(id)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    #68 lvl check
    level = soup.find('p', {'class': "text-yellow"})
    if level == None:
        return None
    else:
        level_str = level.prettify().split('\n')[1]
        level_int = int(level_str[-2:])
        if level_int < 20:
            return None
    
    script_all = soup.findAll('script', type='application/javascript')
    
    user_stats = script_all[-1].prettify()
    
    start_with_items = user_stats.find('USER_ITEMS')
    
    start_with = user_stats.find('{',start_with_items)
    end_with = user_stats.find('}',start_with_items)
    cut_user_items = user_stats[start_with:end_with+1]
    cut_user_items = cut_user_items.replace('null', '0' )
    try:
        dict_of_stats = ast.literal_eval(cut_user_items)
    except ValueError:
        print(id)
        return None
    waste_columns = ['upgrade-','skin-weapon-','sl-','profile-badge-',"quest-"]
    
    unique_skins = ["skin-weapon-m4a1-desert-gum","skin-weapon-an94-male","skin-weapon-aswal-valentine",
                    "skin-weapon-g3-arsen","skin-instrument-guitar-halloween"]
    unique_forms = ["sl-skin-emg","sl-skin-fs","sl-skin-vgs","sl-skin-hunter","sl-skin-omon", 
                    "sl-skin-pumpkin", "sl-skin-frost" ,"sl-skin-xmas", "sl-skin-doctor","sl-skin-mariachi","sl-skin-2cher"
                   "sl-skin-fsg","sl-skin-gentleman", "skin-player-new", "sl-skin-clown", "sl-skin-hunt", "sl-skin-spectator",
                   "sl-skin-unk", "sl-pl-anon"]
    unique_weapons = ["sl-pri-pdrc","sl-pri-g36c","sl-pri-g36c2","sl-pri-kiparis","sl-pri-mp7","sl-pri-p91",
                     "sl-pri-six12","sl-pri-svt40","sl-pri-scout", 
                      "sl-pri-water", "sl-pri-paint","sl-pri-snowball-blaster","sl-pri-serf"]
    unique_sec = ["sl-sec-baltiets", "sl-sec-scorpion", "sl-sec-jericho"]
    unique_knifes = ["sl-ml-knife-hardened","sl-ml-knife-woe", "sl-ml-knife-executor", 
                     "sl-ml-knife-premium", "sl-ml-knife-puma","sl-ml-knife-toyhammer","sl-ml-knife-trench"]
    unique_profile = ['profile-badge-halloween-1','profile-badge-halloween-2',"profile-badge-drawbot","profile-badge-a"
                     "profile-badge-bo", "profile-badge-radiation", "profile-badge-skull", "profile-badge-star"]
    final_fp = ["quest-deal-1","quest-expired-goods-9",'quest-first-aid-4',]
    unique = unique_skins + unique_forms + unique_weapons + unique_sec + unique_knifes + unique_profile + final_fp
    future_deleted_columns =[]
    achiev_counter = 0
    for key in dict_of_stats:
        if key.startswith(tuple(waste_columns)):
            if key in tuple(unique):
                pass
            else:
                future_deleted_columns.append(key)
        elif key.startswith('achievement-'):
            achiev_counter += 1
            future_deleted_columns.append(key)
    for key in future_deleted_columns:
        del dict_of_stats[key]
    dict_of_stats['achievement-counter'] = achiev_counter
    dict_of_stats['level'] = level_int
    df_user = pd.DataFrame(dict_of_stats, index=[id])
    return df_user

In [3]:
import numpy as np
n = 1029000
ids = np.arange(1,n)

In [4]:
%%time
import multiprocessing.dummy as multiprocessing

p = multiprocessing.Pool(50)

results = p.map(scrap,ids)
p.close()
p.join()

Wall time: 3h 52min 1s


In [5]:
%%time
user_items = list()
for i in range(1000):
    k = n//1000
    if i == 999:
        user_items.append(pd.concat(results[i*k:-1],sort=False))
    else:
        user_items.append(pd.concat(results[i*k:(i+1)*k],sort=False))

Wall time: 2min 56s


In [6]:
%%time
items_final = pd.concat(user_items,sort = False)

Wall time: 10.2 s


In [7]:
items_final.to_csv('items_final.csv',index = False)